In [2]:
import import_ipynb
from Job import Job
from Vehicle import Vehicle
import importlib
import numpy as np
from typing import Dict, Tuple, Sequence, List, Any
import matplotlib.pyplot as plt
import folium
import pandas as pd 
import datetime

import json

import pprint


In [3]:
morningFileName = 'morning.json'
afternoonFileName = 'afternoon.json'
vroomUrl = 'http://35.213.166.175:3000'

morning_parcel = "2020/1/15 00:00"
until_morning_parcel = "2020/1/16 09:15"

afternoon_parcel = "2020/1/15 09:16"
until_afternoon_parcel = "2020/1/16 1:15"

subangLocation = [101.5531605, 3.0404532000000004]
firstFourHour = [0, 10800]
secondFourHour = [21600, 36000]



In [4]:
def write_file(fileName):

    text_file = open(fileName, "w")
    n = text_file.write(a )
    text_file.close()
    
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)
    

def sendRequest(fileName):

    url = 'http://35.213.166.175:3000'
    #payload = open("request.json")
    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
    r = requests.post(url, data=open(fileName, 'rb'), headers=headers)    

In [5]:
from datetime import datetime

df = pd.read_csv("processed_dhl_data.csv")


# morning_parcel = "01/15/2020 00:00"
# until_morning_parcel = "01/16/2020 09:15"

# afternoon_parcel = "01/16/2020 09:16"
# until_afternoon_parcel = "01/16/2020 1:15"



# morning_parcel_dt = datetime.strptime(morning_parcel, "%m/%d/%Y %H:%M")
# until_morning_parcel_dt = datetime.strptime(until_morning_parcel, "%m/%d/%Y %H:%M")
# afternoon_parcel_dt = datetime.strptime(afternoon_parcel, "%m/%d/%Y %H:%M")
# until_afternoon_parcel_dt = datetime.strptime(until_afternoon_parcel, "%m/%d/%Y %H:%M")

#df = df.loc[datetime.strptime(df["AR dtm"], "%m/%d/%Y %H:%M")  ]

delivery = df.loc[df['Act Base'] == "D"]
pickup = df.loc[df['Act Base'] == "P"]

#no arrival time and delivery task can not be use
delivery = df.dropna(subset = ['AR dtm'])


delivery['AR dtm'] =  pd.to_datetime(delivery['AR dtm'], format='%m/%d/%Y %H:%M')

mask = (delivery['AR dtm'] >= morning_parcel) & (delivery['AR dtm'] <= until_morning_parcel)
dMorning = delivery.loc[mask]
dAfternoon = delivery.loc[~mask]

pickupDelivery = delivery.append(pickup, ignore_index=True)

pickupDeliveryMorning = dMorning.append(pickup, ignore_index=True)
pickupDeliveryAfternoon = dAfternoon.append(pickup, ignore_index=True)

courierName = df["Courier id"].unique()
zip = df["zip"].unique()
vehiclesNumber = len(courierName)

#afternoon
#zip.astype(int)
#pickupDeliveryAfternoon
#dMorning.to_csv("morning_data.csv")

C:\Users\lun\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
vehicleList = []

for x in range(0, vehiclesNumber):
    vehicleList.append(Vehicle(x+1,subangLocation,[99999], zip , firstFourHour))    

# for x in range(0, vehiclesNumber):
#     vehicleList.append(Vehicle(x + len(vehiclesNumber) + 1,[101.5531605, 3.0404532000000004],[99999], zip , [21600, 36000]))  

In [7]:
vehicleList[0].id

1

In [8]:
jobList = []
#test = Job(1,[1],[1.98935, 48.701],[1], [[32400, 36000]])
#len(pickupDeliveryMorning)
for x in range(0, len(pickupDeliveryMorning)):
    jobList.append(Job(x+1,[1],[pickupDeliveryMorning.iloc[x]["lgtd"], pickupDeliveryMorning.iloc[x]["lat"]] , [pickupDeliveryMorning.iloc[x]["zip"]], [], 300 , pickupDeliveryMorning.iloc[x]["Street"]))

In [9]:
jobList[0].location
pickupDeliveryMorning.iloc[0]["Street"]

'KEMUN NO 11 35/132 JALAN RAJA LUMU BUKIT\r\n40470 SHAH ALAM - SELANGOR\r\nSHAH ALAM'

In [10]:
jobList[0].description

'KEMUN NO 11 35/132 JALAN RAJA LUMU BUKIT\r\n40470 SHAH ALAM - SELANGOR\r\nSHAH ALAM'

In [11]:
test = []
dict_t = {}
dict_t["jobs"] = []
for x in jobList:
    dict_t["jobs"].append(x.__dict__)

dict_t["vehicles"] = []
for y in vehicleList:
    #test.append(json.dumps(x.__dict__))
    dict_t["vehicles"].append(y.__dict__)
    

dict_t

{'jobs': [{'id': 1,
   'delivery': [1],
   'location': [101.52341390000001, 3.0171031000000004],
   'skills': [40470],
   'service': 300,
   'description': 'KEMUN NO 11 35/132 JALAN RAJA LUMU BUKIT\r\n40470 SHAH ALAM - SELANGOR\r\nSHAH ALAM'},
  {'id': 2,
   'delivery': [1],
   'location': [101.52159509999998, 3.012378],
   'skills': [40400],
   'service': 300,
   'description': 'LOT 1298JALAN BUKIT KEMUNING\r\nSECTION 34,40400 SHAH\r\nSHAH ALAM'},
  {'id': 3,
   'delivery': [1],
   'location': [101.5165286, 3.0054585],
   'skills': [40460],
   'service': 300,
   'description': 'LOT 1930 BT 7 JLN BKT KEMUNING\r\nSEC 32\r\nSELANGOR'},
  {'id': 4,
   'delivery': [1],
   'location': [101.5160936, 3.0052323],
   'skills': [40460],
   'service': 300,
   'description': '40460 SHAH ALAM, SELANGOR DARUL EHS\r\nLOT 1929, JALAN BUKIT KEMUNING, SEK\r\nSHAH ALAM'},
  {'id': 5,
   'delivery': [1],
   'location': [101.5144261, 3.007725],
   'skills': [40460],
   'service': 300,
   'description': 'LO

In [12]:
a = json.dumps(dict_t, cls=NpEncoder) 

# b=json.dumps(dict_t)

write_file(morningFileName)
pprint.pprint(json.loads(a))

# dict_t['vehicles']

#json.dumps(dict_t)

{'jobs': [{'delivery': [1],
           'description': 'KEMUN NO 11 35/132 JALAN RAJA LUMU BUKIT\r\n'
                          '40470 SHAH ALAM - SELANGOR\r\n'
                          'SHAH ALAM',
           'id': 1,
           'location': [101.52341390000001, 3.0171031000000004],
           'service': 300,
           'skills': [40470]},
          {'delivery': [1],
           'description': 'LOT 1298JALAN BUKIT KEMUNING\r\n'
                          'SECTION 34,40400 SHAH\r\n'
                          'SHAH ALAM',
           'id': 2,
           'location': [101.52159509999998, 3.012378],
           'service': 300,
           'skills': [40400]},
          {'delivery': [1],
           'description': 'LOT 1930 BT 7 JLN BKT KEMUNING\r\n'
                          'SEC 32\r\n'
                          'SELANGOR',
           'id': 3,
           'location': [101.5165286, 3.0054585],
           'service': 300,
           'skills': [40460]},
          {'delivery': [1],
           'descript

                          'SHAH ALAM',
           'id': 129,
           'location': [101.54586579999999, 3.0581563999999997],
           'service': 300,
           'skills': [40300]},
          {'delivery': [1],
           'description': 'LOT 2, PERSIARAN TENGKU AMPUAN,\r\n'
                          'SECTION 21, SHAH ALAM INDUSTRIAL SI\r\n'
                          'SHAH ALAM',
           'id': 130,
           'location': [101.5428634, 3.0597057999999997],
           'service': 300,
           'skills': [40300]},
          {'delivery': [1],
           'description': 'LOT 2, PERSIARAN TENGKU AMPUAN\r\n'
                          'SEC 21, SHAH ALAM INDUSTRIAL ESTATE\r\n'
                          'SHAH ALAM',
           'id': 131,
           'location': [101.5463311, 3.0595107999999995],
           'service': 300,
           'skills': [40300]},
          {'delivery': [1],
           'description': 'NO.7B PERSIARAN JUBLI PERAK\r\n'
                          'SEKSYEN 22 40 SELANGOR\r\n'


          {'delivery': [1],
           'description': 'LOT 32589,KAWASAN PERUSAHAAN\r\n'
                          'BUKIT SERDANG, SERI KEMBANGAN 40 SELANGOR\r\n'
                          'SERI KEMBANGAN',
           'id': 298,
           'location': [101.69494370000001, 3.0317564999999997],
           'service': 300,
           'skills': [43300]},
          {'delivery': [1],
           'description': 'A 05 09 JALAN SK 13 15 PANGSAPURI\r\n'
                          'KIAMBANG SERI KEMBANGAN SELANGOR\r\n'
                          'KALAU LUMPUR',
           'id': 299,
           'location': [101.6988906, 3.0259443999999998],
           'service': 300,
           'skills': [43300]},
          {'delivery': [1],
           'description': '47-2, JLN BS 1/2\r\n'
                          'OLIVE HILL BUSINESS PARK 40 SELANGOR\r\n'
                          'SERI KEMBANGAN',
           'id': 300,
           'location': [101.6969875, 3.0132453999999997],
           'service': 300,
           '

           'location': [101.5895696, 3.0736177000000002],
           'service': 300,
           'skills': [47500]},
          {'delivery': [1],
           'description': 'SUBANG JAYA BRANCH\r\n'
                          'B1-B4, JALAN SS15/4D\r\n'
                          'SUBANG JAYA',
           'id': 467,
           'location': [101.58660959999999, 3.0735015],
           'service': 300,
           'skills': [47500]},
          {'delivery': [1],
           'description': '35-3 JALAN SS 15/8A\r\nSUBANG JAYA\r\nSUBANG JAYA',
           'id': 468,
           'location': [101.5890069, 3.0740411],
           'service': 300,
           'skills': [47500]},
          {'delivery': [1],
           'description': 'NO : 55 - 2, JALAN SS 15 / 8A,\r\n'
                          'SUBANG JAYA 47500 40 SELANGOR\r\n'
                          'SUBANG JAYA',
           'id': 469,
           'location': [101.5892446, 3.0741725],
           'service': 300,
           'skills': [47500]},
          {'deli

          {'delivery': [1],
           'description': '5 JALAN BU11/4 BANDAR UTAMA\r\n'
                          'PETALING JAYA SELANGOR 47800 MALAYSIA\r\n'
                          'NULL',
           'id': 670,
           'location': [101.6052141, 3.1309139],
           'service': 300,
           'skills': [47800]},
          {'delivery': [1],
           'description': '61 JALAN BU 11/6 SELANGOR\r\n'
                          'PETALING JAYA 47800\r\n'
                          'N',
           'id': 671,
           'location': [101.603372, 3.1297811],
           'service': 300,
           'skills': [47800]},
          {'delivery': [1],
           'description': 'NO.65 JALAN BU11/7\r\n'
                          'BANDAR UTAMA 11/7\r\n'
                          'PETALING JAYA',
           'id': 672,
           'location': [101.6035838, 3.129244],
           'service': 300,
           'skills': [47800]},
          {'delivery': [1],
           'description': '52 JALAN BU 11/7 BANDAR UTA

...

           'service': 300,
           'skills': [47610]},
          {'delivery': [1],
           'description': 'LVL 8, 2 JLN SS21/37, UPTOWN2 DAMANSARA UTAMA',
           'id': 2563,
           'location': [101.622486, 3.1370637],
           'service': 300,
           'skills': [47400]},
          {'delivery': [1],
           'description': 'LSCA DEPT UNIT 201,LVL2 UPTOWN 2 2 JLN SS21/37 '
                          'D`SARA UPTOWN.',
           'id': 2564,
           'location': [101.622486, 3.1370637],
           'service': 300,
           'skills': [47400]},
          {'delivery': [1],
           'description': 'LEVEL 7 MENARA KLK NO 1 JLN PJU 7/6 MUTIARA '
                          'DAMANSARA PETALING JAYA SELANGOR DARUL EHSAN',
           'id': 2565,
           'location': [101.61272890000001, 3.1601071],
           'service': 300,
           'skills': [47800]},
          {'delivery': [1],
           'description': 'LOT 6,JALAN TEKNOLOGI 3/6, TAMAN SAINS SELANGOR 1, '
             

                          62300,
                          62150,
                          68100,
                          51200,
                          55100,
                          48000,
                          53100,
                          52200,
                          52100,
                          47830,
                          52000,
                          53300,
                          50200,
                          47180,
                          47190,
                          47160,
                          47110,
                          47140,
                          47150,
                          47120,
                          47130,
                          40160,
                          45620,
                          45600,
                          42300,
                          48020,
                          48009,
                          44300,
                          48200,
                          48300],
         

                          47400,
                          47800,
                          60000,
                          47820,
                          47810,
                          47301,
                          47410,
                          46410,
                          47000,
                          50000,
                          41300,
                          41050,
                          41400,
                          41150,
                          41000,
                          42100,
                          42200,
                          40100,
                          45800,
                          141840,
                          40170,
                          40802,
                          41700,
                          43650,
                          40450,
                          40675,
                          47200,
                          47300,
                          46100,
                          46300,
         

                          52200,
                          52100,
                          47830,
                          52000,
                          53300,
                          50200,
                          47180,
                          47190,
                          47160,
                          47110,
                          47140,
                          47150,
                          47120,
                          47130,
                          40160,
                          45620,
                          45600,
                          42300,
                          48020,
                          48009,
                          44300,
                          48200,
                          48300],
               'start': [101.5531605, 3.0404532000000004],
               'time_window': [0, 10800]},
              {'capacity': [99999],
               'id': 55,
               'skills': [40470,
                          40400,
           

                          59100,
                          50603,
                          50680,
                          46710,
                          46700,
                          43600,
                          62502,
                          62604,
                          62530,
                          62510,
                          62000,
                          62596,
                          62592,
                          62602,
                          62100,
                          62618,
                          62632,
                          62200,
                          62250,
                          62300,
                          62150,
                          68100,
                          51200,
                          55100,
                          48000,
                          53100,
                          52200,
                          52100,
                          47830,
                          52000,
          

In [13]:
a

'{"jobs": [{"id": 1, "delivery": [1], "location": [101.52341390000001, 3.0171031000000004], "skills": [40470], "service": 300, "description": "KEMUN NO 11 35/132 JALAN RAJA LUMU BUKIT\\r\\n40470 SHAH ALAM - SELANGOR\\r\\nSHAH ALAM"}, {"id": 2, "delivery": [1], "location": [101.52159509999998, 3.012378], "skills": [40400], "service": 300, "description": "LOT 1298JALAN BUKIT KEMUNING\\r\\nSECTION 34,40400 SHAH\\r\\nSHAH ALAM"}, {"id": 3, "delivery": [1], "location": [101.5165286, 3.0054585], "skills": [40460], "service": 300, "description": "LOT 1930 BT 7 JLN BKT KEMUNING\\r\\nSEC 32\\r\\nSELANGOR"}, {"id": 4, "delivery": [1], "location": [101.5160936, 3.0052323], "skills": [40460], "service": 300, "description": "40460 SHAH ALAM, SELANGOR DARUL EHS\\r\\nLOT 1929, JALAN BUKIT KEMUNING, SEK\\r\\nSHAH ALAM"}, {"id": 5, "delivery": [1], "location": [101.5144261, 3.007725], "skills": [40460], "service": 300, "description": "LOT 1929 JALAN KEMUNING,\\r\\nSEKSYEN 32\\r\\nSHAH ALAM"}, {"id": 6,

In [2]:
import requests
import json
import pprint

url = 'http://35.213.166.175:3000'
#payload = open("request.json")
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
#r = requests.post(url, data=open(morningFileName, 'rb'), headers=headers)
# r = requests.post(url, data=open('morning.json', 'rb'), headers=headers)    
r = requests.post(url, data=open('morning.json', 'rb'), headers=headers)    


    
#sendRequest(morningFileName)

In [3]:

r

<Response [200]>

In [4]:
a =r.content

# write_file("response.txt")
# pprint.pprint(json.loads(r.content))
r.content

b'{"code":0,"summary":{"cost":272133,"unassigned":577,"delivery":[2065],"amount":[2065],"pickup":[0],"service":619500,"duration":272133,"waiting_time":0,"distance":2726007,"computing_times":{"loading":13991,"solving":199250,"routing":554}},"unassigned":[{"id":2642,"location":[101.5534596,3.3097056000000006]},{"id":2631,"location":[101.606814,3.2012601000000004]},{"id":2630,"location":[101.6048219,3.2019006]},{"id":2629,"location":[101.6982386,3.2290927000000004]},{"id":2628,"location":[101.7917012,2.9418034]},{"id":2624,"location":[101.372653,3.0318901]},{"id":2623,"location":[101.372653,3.0318901]},{"id":2622,"location":[101.372653,3.0318901]},{"id":2621,"location":[101.372653,3.0318901]},{"id":2620,"location":[101.3712055,3.031718]},{"id":2619,"location":[101.3725818,3.0273437000000005]},{"id":2618,"location":[101.3725818,3.0273437000000005]},{"id":2569,"location":[101.3207856,3.1755399]},{"id":2567,"location":[101.561259,3.1940411]},{"id":2559,"location":[101.719422,3.0182529]},{"id

In [11]:
test = json.loads(r.content)
test

{'code': 0,
 'summary': {'cost': 272133,
  'unassigned': 577,
  'delivery': [2065],
  'amount': [2065],
  'pickup': [0],
  'service': 619500,
  'duration': 272133,
  'waiting_time': 0,
  'distance': 2726007,
  'computing_times': {'loading': 13991, 'solving': 199250, 'routing': 554}},
 'unassigned': [{'id': 2642, 'location': [101.5534596, 3.3097056000000005]},
  {'id': 2631, 'location': [101.606814, 3.2012601000000003]},
  {'id': 2630, 'location': [101.6048219, 3.2019006]},
  {'id': 2629, 'location': [101.6982386, 3.2290927000000003]},
  {'id': 2628, 'location': [101.7917012, 2.9418034]},
  {'id': 2624, 'location': [101.372653, 3.0318901]},
  {'id': 2623, 'location': [101.372653, 3.0318901]},
  {'id': 2622, 'location': [101.372653, 3.0318901]},
  {'id': 2621, 'location': [101.372653, 3.0318901]},
  {'id': 2620, 'location': [101.3712055, 3.031718]},
  {'id': 2619, 'location': [101.3725818, 3.0273437000000003]},
  {'id': 2618, 'location': [101.3725818, 3.0273437000000003]},
  {'id': 2569,

In [15]:
routes = test['routes']
vehicle =  routes

route_1 = []

#print(test["routes"][0]['steps'])
#pprint.pprint(test["routes"][0]['steps'])
index = -1;
for x in vehicle[0]['steps']:
    index+=1
    if(index != 0):
        route_1.append(x['location'])
    else:
        route_1.append(list(reversed(x['location'])))
    
route_new = []
# for z in route_1:
#     route_new.append(list(reversed(z)))
    
index = -1;
for x in vehicle[0]['steps']:
    index+=1
    if(index != 0):
        route_new.append(list(reversed(x['location'])))
    else:
        route_new.append(x['location'])

geometries = []
for i in test['routes']:
    geometries.append(i['geometry'])
test['routes'][1]['geometry']
geometries
# vehicle[0]['steps']
# x

type(r.content)

# f = open('bytes', 'wb')
# f.write(r.content)
# f.close()
pointer = open('bytes', 'rb')
data = pointer.read()
pointer.close()
data

try_read = json.loads(data)
# try_read

In [16]:
unassigned_jobs = test['unassigned']
unassigned_jobs[0]['location']

[101.5534596, 3.3097056000000005]

In [17]:
jobList = []
#test = Job(1,[1],[1.98935, 48.701],[1], [[32400, 36000]])
#len(pickupDeliveryMorning)

def getMorningStreetWithID(id):
    return pickupDeliveryMorning.iloc[id]["Street"]

def getMorningZipWithID(id):
    return pickupDeliveryMorning.iloc[id]["zip"]

for x in range(0, len(unassigned_jobs)):
    jobList.append(Job(unassigned_jobs[x]['id'],[1], [unassigned_jobs[x]["location"][0], unassigned_jobs[x]["location"][1]] , [getMorningZipWithID()], [], 300 , getMorningStreetWithID() ))

for x in range(0, len(pickupDeliveryAfternoon)):
    jobList.append(Job(x+len(pickupDeliveryMorning),[1],[pickupDeliveryAfternoon.iloc[x]["lgtd"], pickupDeliveryAfternoon.iloc[x]["lat"]] , [pickupDeliveryAfternoon.iloc[x]["zip"]], [], 300 , pickupDeliveryAfternoon.iloc[x]["Street"]))
    
test = []
dict_t = {}
dict_t["vehicles"] = []
dict_t["jobs"] = []
for x in vehicleList:
    #test.append(json.dumps(x.__dict__))
    dict_t["vehicles"].append(x.__dict__)
    
for x in jobList:
    dict_t["jobs"].append(x.__dict__)
    
a = json.dumps(dict_t, cls=NpEncoder) 




a

NameError: name 'Job' is not defined

In [ ]:
text_file = open("afternoon.json", "w")
n = text_file.write(a )
text_file.close()

In [ ]:
import requests
import json
import pprint

url = 'http://35.213.166.175:3000'
#payload = open("request.json")
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=open('afternoon.json', 'rb'), headers=headers)